In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Create brower location to pull 

In [3]:
#Website searching Data Analyst positions
indeed = "https://www.indeed.com/jobs?q=data+analyst"

browser.visit(indeed)



In [4]:
html = browser.html
html[0:500]

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Data Analyst Jobs, Employment | Indeed.com</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="28,333 Data Analyst jobs available on Indeed.com. Apply to Data Analyst, Entry Level Data Analyst, Senior Data Analyst and more!">\n    <meta name="keywords" content="Data Analyst Jobs, Em'

In [5]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [ ]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
# jobs = soup.find_all('div', {'class': 'cardOutline'})
# len(jobs)


In [6]:
#Create for loops 
rows = []
browser.visit(indeed)
for i in range(1, 50):
    
    html = browser.html
    soup = BeautifulSoup(browser.html, "html.parser")
    jobs = soup.find_all('div', {'class': 'cardOutline'})
    
    #Within the Jobs for loop we are pulling the job title and link , we added the beginning of the website in our code to correct the format.
    for job in jobs:
        title = job.find('a', {"class": 'jcs-JobTitle'}).text
        link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]
        #Within the Company for loop we had to include an if statement due to some job listings not having a company.
        company_name = job.find('span', {'class': 'companyName'})
        if company_name: 
            company = company_name.text
        else:
            company = None
        #Within the Company link for loop we also had to include an if statement due to some listings not having a company link.
        company_con = job.find('a', {"class": 'companyOverviewLink'})
        if company_con: 
            company_link = "www.indeed.com" + company_con["href"]
        else:
            company_link = None
        #Within the rating for loop we had to include an if statement due to some listings not having a rating.
        rating_con = job.find('span', {"class": 'ratingNumber'})
        if rating_con: 
            rating = rating_con.text
        else:
            rating = None

        #We found the location of the jobs
        loc = job.find('div', {"class": 'companyLocation'}).text
        
        #Within the Salary for loop we had to include an if statement due to some listings not showing the salary for the position
        salary_con = job.find('div', {"class": 'salary-snippet-container'})
        if salary_con: 
            salary = salary_con.text
        else:
            salary = None

        #Within the status we are finding the employment status of the position, we had to include an if statement due to some listings not showing the employment status.
        status_con = job.find('svg', {"aria-label": "Job type"})
        if status_con:
            status = status_con.parent.text
        else:
            status = None

        data = {}
        data["job_title"] = title
        data["job_link"] = link
        data["company"] = company
        data["company_link"] = company_link
        data["company_rating"] = rating
        data["location"] = loc
        data["salary"] = salary
        data["status"] = status
        rows.append(data)
    
    browser.links.find_by_text(f"{i+1}").click()
        
df = pd.DataFrame(rows)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,Customer Service Data Analyst/Support Services...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CK...,VSP Technologies,www.indeed.com/cmp/Vsp-Technologies,3.1,"Prince George, VA 23875",None,Full-time
1,Actimize Business Analyst - US Remote,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CM...,MoneyGram,www.indeed.com/cmp/Moneygram,3.7,"Remote in Dallas, TX 75206","$90,000 - $107,000 a year Employer est.",Full-time
2,Data Analyst Programming Specialist,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CD...,National Indemnity Company,www.indeed.com/cmp/National-Indemnity-Company,4.1,"Omaha, NE",None,Full-time
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,Full-time
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,Full-time


In [7]:
len(df)

735

In [8]:
df["company_rating"] = pd.to_numeric(df.company_rating)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    536 non-null    object 
 4   company_rating  536 non-null    float64
 5   location        735 non-null    object 
 6   salary          359 non-null    object 
 7   status          562 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


In [9]:
#combine with regex s=space d=catch or match all numbers \+ = plus sign matches plus sign, regex meaning.
df["status"] = df.status.str.replace("\s\+\d","")

C:\Users\19722\AppData\Local\Temp\ipykernel_3488\152645899.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["status"] = df.status.str.replace("\s\+\d","")


In [10]:
df.status.value_counts()

Full-time     467
Contract       84
Part-time       5
Temporary       3
Seasonal        2
Internship      1
Name: status, dtype: int64

In [11]:
#Status column -  ID, name, last_updated
df.status.unique()

array(['Full-time', None, 'Contract', 'Internship', 'Temporary',
       'Part-time', 'Seasonal'], dtype=object)

In [12]:
#Status lookup table
status = df.status.dropna().unique()
status_df = pd.DataFrame()
status_df["status"] = status
status_df

,status
0,Full-time
1,Contract
2,Internship
3,Temporary
4,Part-time
5,Seasonal


In [13]:
#Company lookup table, revised to drop the 2nd Insight Global below
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

VSP Technologies                    1
Christus Health                     1
iflow                               1
Citi                                1
Coahoma Community College           1
                                   ..
Bioderm Inc.                        1
Michael Kors                        1
Somatus                             1
Trinity Technology Solutions LLC    1
Mace                                1
Name: company, Length: 626, dtype: int64

In [14]:
companies.loc[companies.company == "Insight Global"]

,company,company_link,company_rating
78,Insight Global,www.indeed.com/cmp/Insight-Global,3.9


In [15]:
df.loc[df.company == "Insight Global", "company_link"] = "www.indeed.com/cmp/Insight-Global"
df.loc[df.company == "Insight Global", "company_rating"] = 3.9

In [16]:
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

VSP Technologies                    1
Christus Health                     1
iflow                               1
Citi                                1
Coahoma Community College           1
                                   ..
Bioderm Inc.                        1
Michael Kors                        1
Somatus                             1
Trinity Technology Solutions LLC    1
Mace                                1
Name: company, Length: 626, dtype: int64

In [17]:
#Companies lookup table
companies

,company,company_link,company_rating
0,VSP Technologies,www.indeed.com/cmp/Vsp-Technologies,3.1
1,MoneyGram,www.indeed.com/cmp/Moneygram,3.7
2,National Indemnity Company,www.indeed.com/cmp/National-Indemnity-Company,4.1
3,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6
4,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0
...,...,...,...
622,MAGRABBIT,None,NaN
623,Massachusetts General Hospital(MGH),www.indeed.com/cmp/Massachusetts-General-Hospital,4.2
624,Center for Sports Medicine & Orthopaedics,www.indeed.com/cmp/Center-For-Sports-Medicine-...,4.1
625,Michaels,www.indeed.com/cmp/Michaels,3.5


In [18]:
#State 
def getState(loc):
    if "," in loc:
        state = loc.replace("\xa0", " ").split(",")[1].strip().split(" ")[0].strip()
    else:
        state = None
        
    return(state)

In [19]:
#Find the locations with"Downtown area"
getState("Austin, TX (Downtown area)")

'TX'

In [20]:
#
df["state"] = df.location.apply(lambda x: getState(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    536 non-null    object 
 4   company_rating  536 non-null    float64
 5   location        735 non-null    object 
 6   salary          359 non-null    object 
 7   status          562 non-null    object 
 8   state           527 non-null    object 
dtypes: float64(1), object(8)
memory usage: 51.8+ KB


In [21]:
#
df.state.value_counts()

TX    236
CA     60
NY     45
IL     17
GA     15
VA     14
PA     13
FL     12
WA     10
MN      9
CO      9
NC      8
OK      8
UT      7
AZ      5
MA      5
TN      5
MD      4
DC      4
NJ      3
MI      3
MS      3
CT      3
OH      3
NE      2
KY      2
OR      2
IN      2
AL      2
NV      2
DE      2
PR      1
NM      1
RI      1
VT      1
WI      1
AR      1
SD      1
KS      1
MO      1
SC      1
IA      1
LA      1
Name: state, dtype: int64

State lookup table

In [22]:
#State look-up table
state = df.state.dropna().unique()
state_df = pd.DataFrame()
state_df["state"] = state
state_df

,state
0,VA
1,TX
2,NE
3,IL
4,MD
5,GA
6,CA
7,NV
8,NY
9,WI


In [23]:
#Find Remote job
def getRemote(loc):
    if "remote" in loc.lower():
        return(True)
    else:
        return(False)
        
  

In [24]:
#
df["is_remote"] = df.location.apply(lambda x: getRemote(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    536 non-null    object 
 4   company_rating  536 non-null    float64
 5   location        735 non-null    object 
 6   salary          359 non-null    object 
 7   status          562 non-null    object 
 8   state           527 non-null    object 
 9   is_remote       735 non-null    bool   
dtypes: bool(1), float64(1), object(8)
memory usage: 52.5+ KB


In [25]:
df.is_remote.value_counts()

True     374
False    361
Name: is_remote, dtype: int64

In [26]:
df.is_remote.info()

<class 'pandas.core.series.Series'>
RangeIndex: 735 entries, 0 to 734
Series name: is_remote
Non-Null Count  Dtype
--------------  -----
735 non-null    bool 
dtypes: bool(1)
memory usage: 863.0 bytes


In [27]:
is_remote = df.is_remote.dropna().unique()
is_remote_df = pd.DataFrame()
is_remote_df["is_remote"] = is_remote
is_remote_df

,is_remote
0,False
1,True


In [28]:
#Find the companies paying for ads
def getAd(link):
    if "pagead" in link.lower():
        return(True)
    else:
        return(False)

In [29]:
df["is_ad"] = df.job_link.apply(lambda x: getAd(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    536 non-null    object 
 4   company_rating  536 non-null    float64
 5   location        735 non-null    object 
 6   salary          359 non-null    object 
 7   status          562 non-null    object 
 8   state           527 non-null    object 
 9   is_remote       735 non-null    bool   
 10  is_ad           735 non-null    bool   
dtypes: bool(2), float64(1), object(8)
memory usage: 53.2+ KB


In [30]:
df.is_ad.value_counts()

False    623
True     112
Name: is_ad, dtype: int64

In [31]:
#Paying for Ads, add to main job table as bool
df.groupby("company").is_ad.sum().sort_values(ascending=False)

company
Insight Global                         5
CyberCoders                            3
Cancer Treatment Centers of America    3
Uber                                   2
Computer Enterprises, Inc. (CEI)       2
                                      ..
Fidelity Investments                   0
Federal Reserve Bank of Richmond       0
Federal Reserve Bank of New York       0
Fannie Mae                             0
pulseData                              0
Name: is_ad, Length: 626, dtype: int64

In [32]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [33]:
USERNAME = "postgres"
PASSWORD = "fAr3st!290"
URL = "localhost"
PORT = 5432
DATABASE = "Jobs_db"

connection_str = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{URL}:{PORT}/{DATABASE}"
#print(connection_str)

engine = create_engine(connection_str)

In [34]:
# create the inspector and connect to the engine
inspector_gadget = inspect(engine)

tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

status
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('status_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'status', 'type': VARCHAR(length=20), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

state
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('state_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'state', 'type': VARCHAR(length=10), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

company
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('company_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'company', 'type': VARCHA

In [35]:
status_df.head()

,status
0,Full-time
1,Contract
2,Internship
3,Temporary
4,Part-time


In [36]:
engine.execute("truncate table status restart identity cascade")
status_df.to_sql("status", engine, method="multi", if_exists = "append", index=False)

6

In [37]:
engine.execute("truncate table state restart identity cascade")
state_df.to_sql("state", engine, method="multi", if_exists = "append", index=False)

43

In [38]:
#Drop one company with no name, inclue in writeup
companies = companies.loc[pd.notnull(companies.company)].reset_index(drop=True)
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         626 non-null    object 
 1   company_link    442 non-null    object 
 2   company_rating  442 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.8+ KB


In [39]:
engine.execute("truncate table company restart identity cascade")
companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

626

In [ ]:
df.head()

In [40]:
status2 = pd.read_sql("select id as status_id, status from status", engine)
status2.head(20)

,status_id,status
0,1,Full-time
1,2,Contract
2,3,Internship
3,4,Temporary
4,5,Part-time
5,6,Seasonal


In [41]:
df = pd.merge(df, status2, on = "status", how ="left")
df.drop("status", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,Customer Service Data Analyst/Support Services...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CK...,VSP Technologies,www.indeed.com/cmp/Vsp-Technologies,3.1,"Prince George, VA 23875",None,VA,False,True,1.0
1,Actimize Business Analyst - US Remote,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CM...,MoneyGram,www.indeed.com/cmp/Moneygram,3.7,"Remote in Dallas, TX 75206","$90,000 - $107,000 a year Employer est.",TX,True,True,1.0
2,Data Analyst Programming Specialist,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CD...,National Indemnity Company,www.indeed.com/cmp/National-Indemnity-Company,4.1,"Omaha, NE",None,NE,False,True,1.0
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,TX,False,True,1.0
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,IL,False,True,1.0


In [43]:
state2 = pd.read_sql("select id as state_id, state from state", engine)
df = pd.merge(df, state2, on = "state", how ="left")
df.drop("state", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,is_remote,is_ad,status_id,state_id
0,Customer Service Data Analyst/Support Services...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CK...,VSP Technologies,www.indeed.com/cmp/Vsp-Technologies,3.1,"Prince George, VA 23875",None,False,True,1.0,1.0
1,Actimize Business Analyst - US Remote,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CM...,MoneyGram,www.indeed.com/cmp/Moneygram,3.7,"Remote in Dallas, TX 75206","$90,000 - $107,000 a year Employer est.",True,True,1.0,2.0
2,Data Analyst Programming Specialist,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CD...,National Indemnity Company,www.indeed.com/cmp/National-Indemnity-Company,4.1,"Omaha, NE",None,False,True,1.0,3.0
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,False,True,1.0,2.0
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,False,True,1.0,4.0


In [44]:
company2 = pd.read_sql("select id as company_id, company from company", engine)
df = pd.merge(df, company2, on = "company", how ="left")
df.drop(["company", "company_link", "company_rating"], axis=1, inplace=True)
df.head()

,job_title,job_link,location,salary,is_remote,is_ad,status_id,state_id,company_id
0,Customer Service Data Analyst/Support Services...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CK...,"Prince George, VA 23875",None,False,True,1.0,1.0,1.0
1,Actimize Business Analyst - US Remote,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CM...,"Remote in Dallas, TX 75206","$90,000 - $107,000 a year Employer est.",True,True,1.0,2.0,2.0
2,Data Analyst Programming Specialist,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CD...,"Omaha, NE",None,False,True,1.0,3.0,3.0
3,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,"Dallas, TX 75229 (Northwest Dallas area)",None,False,True,1.0,2.0,4.0
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,"Chicago, IL 60631 (Norwood Park area)",None,False,True,1.0,4.0,5.0


In [46]:
#Database
df.to_sql("job", engine, method="multi", if_exists = "append", index=False)

Code we used as tests and attempts

In [ ]:
#Don't worry about it
# def getCity(loc):
#     if "," in loc:
#         city_list = loc.replace("\xa0", " ").split(",")[0].strip().split(" ")
#         city = city_list[0] if len(city_list) ==1 else city_list[1]
#     else:
#         city = None
        
#     return(city)

In [ ]:
# df["city"] = df.location.apply(lambda x: getCity(x))
# df.info()

In [ ]:
# df.city.value_counts()

In [ ]:
# df.loc[df.location.str.contains("in")]

In [ ]:
#Create a CSV for companies 
#companies.to_csv('companies.csv', index=False)

In [ ]:
#companies_file = pd.read_csv('companies.csv')
#companies_file

In [ ]:
#Create a CSV for States
#state_df.to_csv('state.csv', index=False)

In [ ]:
#state_file = pd.read_csv('state.csv')
#state_file

In [ ]:
#Create a CSV for status (Full-time/part-time)
#status_df.to_csv('status.csv', index=False)

In [ ]:
#status_file = pd.read_csv('status.csv')
#status_file

In [ ]:
#Set-up sleep function if needed
# browser.visit(dice_lyst)
# time.sleep(2)